# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [3]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [6]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
# from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




# @retry(JSONRPCException, tries=10, delay=1, back_off=2)
# def get_tx_result(_tx_hash):
#     tx_result = icon_service.get_transaction_result(_tx_hash)
#     return tx_result




### Required service

In [7]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [8]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [9]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

4621340952430198414714

## Use this to import default tbears wallet and run the transactions


In [14]:
password = "test1_Account"
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
print(deployer_wallet.get_address())
icon_service.get_balance(deployer_wallet.get_address())

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb


786640639999000000000000000

# Deploying all contracts


In [53]:



params={}

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content("rewards"))\
    .params(params)\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)





In [55]:

res = icon_service.get_transaction_result(tx_hash)
print(res)


{'txHash': '0xd96eba78d9058ed5a59606097a65f923978e5e72758312f06d28744183e51968', 'blockHeight': 11884047, 'blockHash': '0x47cf4a0a467f92b26a0a8882b92d4a17b30536ef65dcdeffc1c4fd0920a8c534', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx7306a71207383643e8b14295d842343e6c7e5c30', 'stepUsed': 1424464480, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1424464480, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Updating the contract

In [35]:

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxa8c33fed68a301347b5f01548db17c2870dab36f")\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content("rewards"))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)



In [8]:
contracts={}
contracts['rewards']=res['scoreAddress']
print(contracts)

{'rewards': 'cx5db9eb9a818aa23ad10830f06b546102e1c96f82'}


## Update Distribution percentage



In [49]:

params = { "_data_source_dict_list":
                [
                  {"data_source_name":"Omm3","bal_token_dist_percent":"1000000000000000000"},
                    {"data_source_name":"Omm2","bal_token_dist_percent":"1000000000000000000"},
                    {"data_source_name":"Omm1","bal_token_dist_percent":"1000000000000000000"}
            
                ]
                
                  
          }

# def update_bal_token_dist_percentage(self, _data_source_dict_list : List[DistPercentDict])
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxa8c33fed68a301347b5f01548db17c2870dab36f") \
    .nid(NID) \
    .nonce(100) \
    .method("update_bal_token_dist_percentage")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)


tx_hash = icon_service.send_transaction(signed_transaction)
import time
time.sleep(5)
res = icon_service.get_transaction_result(tx_hash)
print(res)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32032, 'message': 'Data sources mismatched!'}


JSONRPCException: {'code': -32032, 'message': 'Data sources mismatched!'} (Json rpc error)

## Adding New Data Source

In [41]:

params ={"_data_source_name":"Omm2","_contract_address":"cxdcbae76661c0edbf968d048c3bba3311bde57e39"}
# def add_new_data_source(self, _data_source_dict: DataSourceDict):
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxa8c33fed68a301347b5f01548db17c2870dab36f") \
    .nid(NID) \
    .nonce(100) \
    .method("add_new_data_source")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)


tx_hash = icon_service.send_transaction(signed_transaction)

# get_tx_result(tx_hash)

In [43]:
res = icon_service.get_transaction_result(tx_hash)
print(res)

{'txHash': '0xb5af3c446d2e5416e9808f6f643428187c7af5d98d0246c72e158975b28d6a66', 'blockHeight': 11883843, 'blockHash': '0xef97218a7a0bcf7ec8923d89e414495c793792a5859cc25c750c31880aa1e64d', 'txIndex': 1, 'to': 'cxa8c33fed68a301347b5f01548db17c2870dab36f', 'stepUsed': 160400, 'stepPrice': 10000000000, 'cumulativeStepUsed': 160400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Distribute

In [ ]:

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['rewards']) \
    .nid(NID) \
    .nonce(100) \
    .method("distribute")\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)


tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Get Data Source

In [22]:
params = {'_data_source_name': "Omm3"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxa8c33fed68a301347b5f01548db17c2870dab36f")\
    .method("get_data_sources")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,value in response.items():
    if key == "balTokenDistPercent":
        response[key]=int(response[key],0)/ 10**18


print(response)

{'day': '0x0', 'contract_address': 'cxdcbae76661c0edbf968d048c3bba3311bde57e39', 'balTokenDistPercent': 0.0}


## Get Bal Token Dist Per Day

In [30]:
# def bal_token_dist_per_day(self, _day: int) -> int:

params = {'_day': 1}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['rewards'])\
    .method("bal_token_dist_per_day")\
    .params(params)\
    .build()
response = icon_service.call(_call)

response=int(response,0) / 10**18
print(response)
        

100000.0
